
# Loading binance data into trading universe

Loading CEX data into the TradingStrategy affords you the ability to work with longer historical periods with trading pairs. Using longer historical periods allows you to train your model on more data, which can lead to better results. The only downside is that you will have to wait longer for your backtests to complete. 

In [1]:
# from dotenv import load_dotenv
import os
import datetime

from tradingstrategy.chain import ChainId
from tradingstrategy.timebucket import TimeBucket
from tradeexecutor.state.identifier import AssetIdentifier

from tradingstrategy.chain import ChainId
from tradingstrategy.timebucket import TimeBucket
from tradingstrategy.lending import LendingProtocolType
from tradeexecutor.strategy.cycle import CycleDuration
from tradeexecutor.strategy.strategy_module import StrategyType, TradeRouting, ReserveCurrency


START_AT_DATA = datetime.datetime(2023, 1, 1)
START_AT = datetime.datetime(2023, 2, 1)
END_AT = datetime.datetime(2023, 11, 8)
CHAIN_ID = ChainId.polygon
EXCHANGE_SLUG = 'uniswap-v3'
CANDLE_TIME_BUCKET = TimeBucket.h4

TRADING_STRATEGY_TYPE_ENGINE_VERSION = "0.3"

# What kind of strategy we are running.
# This tells we are going to use
# NOTE: this setting has currently no effect
TRADING_STRATEGY_TYPE = StrategyType.managed_positions

# How our trades are routed.
TRADE_ROUTING = TradeRouting.ignore

# How often the strategy performs the decide_trades cycle.
# We do it for every 4h.
TRADING_STRATEGY_CYCLE = CycleDuration.cycle_4h

# Strategy keeps its cash in USDC
RESERVE_CURRENCY = ReserveCurrency.usdc

# Which lending reserves we are using for supplying/borrowing assets
LENDING_RESERVES = [
    (ChainId.polygon, LendingProtocolType.aave_v3, "WETH"),
    (ChainId.polygon, LendingProtocolType.aave_v3, "USDC"),
]

BASE_TOKEN_SYMBOL = "WETH"
QUOTE_TOKEN_SYMBOL = "USDC"
PAIR_FEE = 0.0005

# How much % of the available trading capital to put on a single trade
POSITION_SIZE = 0.75

# Start with this amount of USD
INITIAL_DEPOSIT = 50_000

# Candle time granularity we use to trigger stop loss checks
STOP_LOSS_TIME_BUCKET = TimeBucket.h1


# How many candles we load in the decide_trades() function for calculating indicators
LOOKBACK_WINDOW = 20

# Exponential Moving Average (EMA)
# How many candles to smooth out for EMA line
EMA_CANDLE_COUNT = 2  


# How many candles we use to calculate the Relative Strength Index
RSI_LENGTH = 5

# RSI threshold for opening a long position
RSI_THRESHOLD = 25 

# RSI threshold for opening a short position
RSI_THRESHOLD_SHORT = 75 

# START_AT = df.index[0].to_pydatetime() # datetime.datetime(2022, 1, 1)

# END_AT = df.index[-1].to_pydatetime() # datetime.datetime(2022, 1,18)


### FOR LONGS ###

# Stop Loss relative to the mid price during the time when the position is opened
# If the price drops below this level, trigger a stop loss sell
STOP_LOSS_PCT = 0.985

# Take profit percentage for longs
TAKE_PROFIT_PCT = 1.15

# What is the trailing stop loss level, this trails the most recent candle close price
TRAILING_STOP_LOSS_PCT = 0.99

# Activate trailing stop loss when this level is reached
# TRAILING_STOP_LOSS_ACTIVATION_LEVEL=1.01 (In this strategy we don't use a fixed activation level but EMA line crossing is used instead.)


### FOR SHORTING ###

TAKE_PROFIT_SHORT_PCT = 1.15

STOP_LOSS_SHORT_PCT = 0.985

# What is the Trailing Stop Loss level
TRAILING_STOP_LOSS_SHORT_PCT = 0.99

# Activate Trailing Stop Loss when this level is reached
#TRAILING_STOP_LOSS_SHORT_ACTIVATION_LEVEL = 0.99 (In this strategy we don't use a fixed activation level but EMA line crossing is used instead.)

# Leverage ratio for shorting positions. This is due to the mechanism that the shorting protocol needs as we are not able to borrow assets with 1:1 collateral. 
LEVERAGE = 2


## Creating our assets

In the following code block, we create our ETH and USDC assets, which follow real on-chain deployment addresses. 

In [2]:
# from eth_defi.chain import install_chain_middleware
# from eth_defi.abi import get_deployed_contract
# from tradeexecutor.state.identifier import TradingPairIdentifier

# from web3 import Web3, HTTPProvider

# polygon_json_rpc = os.environ["MY_JSON_RPC_POLYGON"]
# print(polygon_json_rpc)

# web3 = Web3(HTTPProvider(polygon_json_rpc, request_kwargs={"timeout": 5}))
# install_chain_middleware(web3)

# # usdc with $4B supply
# # https://polygonscan.com/address/0x2791bca1f2de4661ed88a30c99a7a9449aa84174
# usdc_token = get_deployed_contract(
#     web3, "ERC20MockDecimals.json", "0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174"
# )

# # eth token.
# # https://polygonscan.com//address/0x7ceb23fd6bc0add59e62ac25578270cff1b9f619
# # https://tradingstrategy.ai/trading-view/polygon/tokens/0x7ceb23fd6bc0add59e62ac25578270cff1b9f619
# eth_token = get_deployed_contract(
#     web3, "ERC20MockDecimals.json", "0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619"
# )

# usdc_asset = AssetIdentifier(
#     CHAIN_ID.value,
#     usdc_token.address,
#     usdc_token.functions.symbol().call(),
#     usdc_token.functions.decimals().call(),
# )

# eth_asset = AssetIdentifier(
#     CHAIN_ID.value,
#     eth_token.address,
#     eth_token.functions.symbol().call(),
#     eth_token.functions.decimals().call(),
# )


# pair = TradingPairIdentifier(
#     base = eth_asset,
#     quote = usdc_asset,
#     pool_address='0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640',
#     exchange_address='0x1F98431c8aD98523631AE4a59f267346ea31F984',
#     internal_id=2854973,
#     fee=0.0005,
#     internal_exchange_id=3990,
#     info_url='https://tradingstrategy.ai/trading-view/polygon/uniswap-v3/eth-usdc-fee-5'
# )

## Download Binance candlestick data

- In the following code block, we show how to download simple candlestick price data from Binance. In this instance, we want 4 hour candles for ETH with the same dates as the strategy backtest.
- Note that we add additional columns to the Binance OHLCV data to make it compatible with our framework.

### Caching

Note: the `get_binance_candlestick_data` already caches data so no need to manually save your own version. If you want to force a redownload, you can set the `force_download` parameter to `True`.

We also examine the dataset to see what it looks like. We can see that the dataset contains the following columns:

- `Date`: The date of the start of the time interval
- `open`: The price of the asset at the start of the time interval
- `high`: The highest price of the asset during the time interval
- `low`: The lowest price of the asset during the time interval
- `close`: The price of the asset at the end of the time interval
- `volume`: The volume of the asset traded during the time interval

In [3]:
import pandas as pd
from tradingstrategy.binance_data import BinanceDownloader
from tradeexecutor.state.identifier import generate_pair_for_binance_data, add_info_columns_to_ohlc

downloader = BinanceDownloader()

pair = generate_pair_for_binance_data(BASE_TOKEN_SYMBOL, QUOTE_TOKEN_SYMBOL, PAIR_FEE)

symbol = "ETHUSDT"

# use stop_loss_time_bucket since, in this case, it's more granular data than the candle_time_bucket
# we later resample to the higher time bucket for the backtest candles
df = downloader.fetch_candlestick_data(
    symbol,
    STOP_LOSS_TIME_BUCKET,
    START_AT_DATA,
    END_AT,
)

df = add_info_columns_to_ohlc(df, pair)

# we need to overwrite the cache for when the candles are loaded later
path = downloader.get_parquet_path(symbol, STOP_LOSS_TIME_BUCKET, START_AT_DATA, END_AT)
df.to_parquet(path)

display(df.head())
display(df.tail())

,open,high,low,close,volume,base_token_symbol,quote_token_symbol,exchange_slug,chain_id,fee,pair_id,buy_volume_all_time,address,exchange_id,token0_address,token1_address,token0_symbol,token1_symbol,token0_decimals,token1_decimals
2023-01-01 00:00:00,1200.09,1201.11,1193.08,1196.19,9549.4793,WETH,USDC,binance,0,5.0,134093847,0,-0x373e22c274add87e,129875571,0x5359e9235cc41cabee78f7f2a4fcf01e650ab475,0x8591ee9090c0c02ca1f103cb637131d8f358870a,WETH,USDC,18,18
2023-01-01 01:00:00,1196.18,1197.43,1193.60,1196.13,5927.4320,WETH,USDC,binance,0,5.0,134093847,0,-0x373e22c274add87e,129875571,0x5359e9235cc41cabee78f7f2a4fcf01e650ab475,0x8591ee9090c0c02ca1f103cb637131d8f358870a,WETH,USDC,18,18
2023-01-01 02:00:00,1196.13,1196.70,1192.72,1194.09,5889.3840,WETH,USDC,binance,0,5.0,134093847,0,-0x373e22c274add87e,129875571,0x5359e9235cc41cabee78f7f2a4fcf01e650ab475,0x8591ee9090c0c02ca1f103cb637131d8f358870a,WETH,USDC,18,18
2023-01-01 03:00:00,1194.09,1196.37,1193.84,1196.02,3157.2079,WETH,USDC,binance,0,5.0,134093847,0,-0x373e22c274add87e,129875571,0x5359e9235cc41cabee78f7f2a4fcf01e650ab475,0x8591ee9090c0c02ca1f103cb637131d8f358870a,WETH,USDC,18,18
2023-01-01 04:00:00,1196.01,1196.74,1194.11,1195.40,3752.0476,WETH,USDC,binance,0,5.0,134093847,0,-0x373e22c274add87e,129875571,0x5359e9235cc41cabee78f7f2a4fcf01e650ab475,0x8591ee9090c0c02ca1f103cb637131d8f358870a,WETH,USDC,18,18


,open,high,low,close,volume,base_token_symbol,quote_token_symbol,exchange_slug,chain_id,fee,pair_id,buy_volume_all_time,address,exchange_id,token0_address,token1_address,token0_symbol,token1_symbol,token0_decimals,token1_decimals
2023-11-08 20:00:00,1892.31,1903.79,1890.75,1900.22,12473.5485,WETH,USDC,binance,0,5.0,134093847,0,-0x373e22c274add87e,129875571,0x5359e9235cc41cabee78f7f2a4fcf01e650ab475,0x8591ee9090c0c02ca1f103cb637131d8f358870a,WETH,USDC,18,18
2023-11-08 21:00:00,1900.23,1904.69,1894.65,1899.07,10545.4582,WETH,USDC,binance,0,5.0,134093847,0,-0x373e22c274add87e,129875571,0x5359e9235cc41cabee78f7f2a4fcf01e650ab475,0x8591ee9090c0c02ca1f103cb637131d8f358870a,WETH,USDC,18,18
2023-11-08 22:00:00,1899.07,1900.98,1890.15,1895.59,8742.9391,WETH,USDC,binance,0,5.0,134093847,0,-0x373e22c274add87e,129875571,0x5359e9235cc41cabee78f7f2a4fcf01e650ab475,0x8591ee9090c0c02ca1f103cb637131d8f358870a,WETH,USDC,18,18
2023-11-08 23:00:00,1895.59,1904.59,1883.01,1886.87,21684.3701,WETH,USDC,binance,0,5.0,134093847,0,-0x373e22c274add87e,129875571,0x5359e9235cc41cabee78f7f2a4fcf01e650ab475,0x8591ee9090c0c02ca1f103cb637131d8f358870a,WETH,USDC,18,18
2023-11-09 00:00:00,1886.87,1896.12,1886.87,1891.09,15703.8793,WETH,USDC,binance,0,5.0,134093847,0,-0x373e22c274add87e,129875571,0x5359e9235cc41cabee78f7f2a4fcf01e650ab475,0x8591ee9090c0c02ca1f103cb637131d8f358870a,WETH,USDC,18,18


## Add rows to dataset and save file

- We need to add rows for the `base_token`, `quote_token` and `exchange`
- We write the DataFrame back to a Parquet file so we don't have to redownload the data every time we run the notebook

In [4]:
from tradingstrategy.exchange import ExchangeUniverse, Exchange, ExchangeType
from tradeexecutor.strategy.pandas_trader.alternative_market_data import load_candle_universe_from_parquet
from tradeexecutor.state.identifier import generate_exchange_for_binance_data

path = downloader.get_parquet_path(symbol, STOP_LOSS_TIME_BUCKET, START_AT_DATA, END_AT)

candle_universe, stop_loss_candle_universe = load_candle_universe_from_parquet(
    pair=pair,
    file=path,
    include_as_trigger_signal=True,
    resample=CANDLE_TIME_BUCKET, 
)

binance_exchange = generate_exchange_for_binance_data(pair)

exchange_universe = ExchangeUniverse.from_collection([binance_exchange])

# TODO how to do for multiple pairs?
pairs_df = candle_universe.pairs.first().reset_index()

display(pairs_df.head())

,pair_id,open,high,low,close,volume,base_token_symbol,quote_token_symbol,exchange_slug,chain_id,...,buy_volume_all_time,address,exchange_id,token0_address,token1_address,token0_symbol,token1_symbol,token0_decimals,token1_decimals,timestamp
0,134093847,1200.09,1201.11,1192.72,1196.02,24523.5032,WETH,USDC,binance,0,...,0,-0x373e22c274add87e,129875571,0x5359e9235cc41cabee78f7f2a4fcf01e650ab475,0x8591ee9090c0c02ca1f103cb637131d8f358870a,WETH,USDC,18,18,2023-01-01


### Create LendingReserveUniverse

This is a description of our lending reserve assets

In [5]:
from tradingstrategy.lending import LendingProtocolType, LendingReserveUniverse

from tradingstrategy.client import Client

client = Client.create_jupyter_client()
universe = client.fetch_lending_reserve_universe()

usdc_desc = (ChainId.polygon, LendingProtocolType.aave_v3, "USDC")
weth_desc = (ChainId.polygon, LendingProtocolType.aave_v3, "WETH")

lending_reserve_universe = universe.limit([usdc_desc, weth_desc])

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /home/alex/.tradingstrategy


### Get binance lending and supply interest rate data

In [6]:
from tradingstrategy.lending import LendingCandleUniverse
from tradingstrategy.binance_data import convert_binance_lending_rates_to_supply

# get IDS for ETH and USDC
eth_reserve_id = lending_reserve_universe.resolve_lending_reserve(weth_desc).reserve_id
usdc_reserve_id = lending_reserve_universe.resolve_lending_reserve(usdc_desc).reserve_id

eth_lending_data = downloader.fetch_lending_rates("ETH", CANDLE_TIME_BUCKET, START_AT_DATA, END_AT)
eth_supply_data = convert_binance_lending_rates_to_supply(eth_lending_data)

usdc_lending_data = downloader.fetch_lending_rates("USDC", CANDLE_TIME_BUCKET, START_AT_DATA, END_AT)
usdc_supply_data = convert_binance_lending_rates_to_supply(usdc_lending_data)

display(eth_lending_data.head()) 
display(eth_supply_data.head())

,lending_rates
2022-12-31 00:00:00,0.000054
2022-12-31 04:00:00,0.000054
2022-12-31 08:00:00,0.000054
2022-12-31 12:00:00,0.000054
2022-12-31 16:00:00,0.000054


,lending_rates
2022-12-31 00:00:00,0.000054
2022-12-31 04:00:00,0.000054
2022-12-31 08:00:00,0.000054
2022-12-31 12:00:00,0.000054
2022-12-31 16:00:00,0.000054


In [7]:
binance_exchange = generate_exchange_for_binance_data(pair)

exchange_universe = ExchangeUniverse.from_collection([binance_exchange])

### Lending candle universe

Finally, we create our lending candle universe, which is a combination of our USDC and ETH lending candle universes.

In [8]:
from tradeexecutor.strategy.trading_strategy_universe import Dataset
from tradingstrategy.lending import convert_interest_rates_to_lending_candle_type_map

lending_candle_type_map = convert_interest_rates_to_lending_candle_type_map({"reserve_id":eth_reserve_id,"lending_data": eth_lending_data, "supply_data":eth_supply_data}, {"reserve_id":usdc_reserve_id,"lending_data": usdc_lending_data, "supply_data":usdc_supply_data})

lending_candle_universe = LendingCandleUniverse(lending_candle_type_map, lending_reserve_universe)

dataset = Dataset(
    time_bucket=CANDLE_TIME_BUCKET,
    exchanges=exchange_universe,
    pairs=pairs_df,
    candles=candle_universe.df,
    backtest_stop_loss_time_bucket=STOP_LOSS_TIME_BUCKET,
    backtest_stop_loss_candles=candle_universe.df,
    lending_candles=lending_candle_universe,
    lending_reserves=lending_reserve_universe,
)

## Create TradingStrategyUniverse

Finally, we create our TradingStrategyUniverse, which is a combination of our TradingStrategy and LendingCandleUniverse.

In [9]:
from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse

pair_ticker = pair.identifier_to_pair_ticker('binance')

universe = TradingStrategyUniverse.create_single_pair_universe(
    dataset=dataset,
    pair=pair_ticker,
)

print(f"We loaded {universe.universe.candles.get_candle_count():,} candles.")

We loaded 1,873 candles.


## Using the universe in a strategy

In [10]:
from typing import List, Dict

from pandas_ta import bbands
from pandas_ta.overlap import ema
from pandas_ta.momentum import rsi

from tradingstrategy.universe import Universe
from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse

from tradeexecutor.state.visualisation import PlotKind, PlotShape
from tradeexecutor.state.trade import TradeExecution
from tradeexecutor.strategy.pricing_model import PricingModel
from tradeexecutor.strategy.pandas_trader.position_manager import PositionManager
from tradeexecutor.state.state import State
from tradeexecutor.strategy.pandas_trader.position_manager import PositionManager


def decide_trades(
        timestamp: pd.Timestamp,
        strategy_universe: TradingStrategyUniverse,
        state: State,
        pricing_model: PricingModel,
        cycle_debug_data: Dict) -> List[TradeExecution]:
  
    universe = strategy_universe.universe

    # We have only a single trading pair for this strategy.
    pair = universe.pairs.get_single()

    # How much cash we have in a hand
    cash = state.portfolio.get_current_cash()

    # Get OHLCV candles for our trading pair as Pandas Dataframe.
    # We could have candles for multiple trading pairs in a different strategy,
    # but this strategy only operates on single pair candle.
    # We also limit our sample size to N latest candles to speed up calculations.
    candles: pd.DataFrame = universe.candles.get_single_pair_data(timestamp, sample_count=LOOKBACK_WINDOW, raise_on_not_enough_data=False)

    # We have data for open, high, close, etc.
    # We only operate using candle close values in this strategy.
    close_prices = candles["close"]

    # Calculate exponential moving from candle close prices
    # https://tradingstrategy.ai/docs/programming/api/technical-analysis/overlap/help/pandas_ta.overlap.ema.html#ema
    ema_series = ema(close_prices, length=EMA_CANDLE_COUNT)
    
    # Calculate RSI from candle close prices
    # https://tradingstrategy.ai/docs/programming/api/technical-analysis/momentum/help/pandas_ta.momentum.rsi.html#rsi
    rsi_series = rsi(close_prices, length=RSI_LENGTH)

    trades = []

    if ema_series is None or rsi_series is None:
        return trades
 
    ema_latest = ema_series.iloc[-1]      # Let's take the latest EMA value from the series
    price_latest = close_prices.iloc[-1]  # Let's take the latest close price value from the series
    current_rsi = rsi_series.iloc[-1]
    
    

    # Create a position manager helper class that allows us easily to create
    # opening/closing trades for different positions
    position_manager = PositionManager(timestamp, strategy_universe, state, pricing_model)

    

    
    ### LONGING ###
    
    stoploss_price_long = None    # We use this to track Stop Loss price for Long positions and draw it to the price chart
    
    if not position_manager.is_any_long_position_open():
        if price_latest < ema_latest and current_rsi < RSI_THRESHOLD and close_prices.iloc[-2] > ema_series.iloc[-2]:
            amount = cash * POSITION_SIZE
            new_trades = position_manager.open_1x_long(pair, amount, stop_loss_pct=STOP_LOSS_PCT, take_profit_pct=TAKE_PROFIT_PCT)
            trades.extend(new_trades)
            stoploss_price_long = position_manager.get_current_long_position().stop_loss
            
    else:
        current_position = position_manager.get_current_long_position()
        # LONGING: We activate trailing stop loss when the price closes above the EMA line.

        if price_latest >= ema_latest:
            # adjust trailing stop loss level for the open long position
            # Stop loss is the only way we sell in this set up, unless TAKE_PROFIT_PCT level has been reached

            current_position.trailing_stop_loss_pct = TRAILING_STOP_LOSS_PCT
            stoploss_price_long = position_manager.get_current_long_position().stop_loss
            if position_manager.get_current_long_position().stop_loss <= float(price_latest * TRAILING_STOP_LOSS_PCT):   # Move the trailing stop loss level only of the new value is higher
                current_position.stop_loss = float(price_latest * TRAILING_STOP_LOSS_PCT)
                stoploss_price_long = position_manager.get_current_long_position().stop_loss

            
    ### SHORTING ###
    
    stoploss_price_short = None   # We use this to track Stop Loss price for Short positions and draw it to the price chart
    
    if not position_manager.is_any_short_position_open():
        # No open positions, decide if open a position in this cycle.
        # We open short if the latest candle has upper wick above BB upper line and close under this line
        
        if price_latest > ema_latest and current_rsi > RSI_THRESHOLD_SHORT and close_prices.iloc[-2] < ema_series.iloc[-2]:
            amount = cash * POSITION_SIZE
            # new_trades = position_manager.open_short(pair, amount, leverage=LEVERAGE, stop_loss_pct=STOP_LOSS_SHORT_PCT, take_profit_pct=TAKE_PROFIT_SHORT_PCT)
            # trades.extend(new_trades)
        
            # stoploss_price_short = position_manager.get_current_short_position().stop_loss

            
    else:
        current_position = position_manager.get_current_short_position()
        # SHORTING: We activate trailing stop loss when the price closes below the EMA line.

        if price_latest <= ema_latest:             
            # adjust trailing stop loss level for the open short position
            # Stop loss is the only way we sell in this set up, unless TAKE_PROFIT_SHORT_PCT level has been reached
            current_position.trailing_stop_loss_pct = TRAILING_STOP_LOSS_SHORT_PCT
            stoploss_price_short = position_manager.get_current_short_position().stop_loss
            if position_manager.get_current_short_position().stop_loss >= float(price_latest * TRAILING_STOP_LOSS_SHORT_PCT):   # Move the trailing stop loss level only of the new value is lower
                current_position.stop_loss = float(price_latest * TRAILING_STOP_LOSS_SHORT_PCT)
                stoploss_price_short = position_manager.get_current_short_position().stop_loss

    any_open_positions = position_manager.is_any_open()
    awe = 100 if any_open_positions else 0

    # Visualise our technical indicators
    visualisation = state.visualisation
    visualisation.plot_indicator(timestamp, "EMA", PlotKind.technical_indicator_on_price, ema_series.iloc[-1], colour="black")
    visualisation.plot_indicator(timestamp, "RSI", PlotKind.technical_indicator_detached, current_rsi)
    visualisation.plot_indicator(timestamp, "RSI Threshold", PlotKind.technical_indicator_overlay_on_detached, RSI_THRESHOLD, detached_overlay_name="RSI")
    #visualisation.plot_indicator(timestamp, "RSI Threshold", PlotKind.technical_indicator_detached, RSI_THRESHOLD_SHORT, detached_overlay_name="RSI")

    visualisation.plot_indicator(timestamp, "Stop Loss long", PlotKind.technical_indicator_on_price, stoploss_price_long, colour="purple", plot_shape=PlotShape.horizontal_vertical)
    visualisation.plot_indicator(timestamp, "Stop Loss short", PlotKind.technical_indicator_on_price, stoploss_price_short, colour="blue", plot_shape=PlotShape.horizontal_vertical)

    return trades

## Create the client

Create the trading strategy client to run the backtest

In [11]:
from tradingstrategy.client import Client

client = Client.create_jupyter_client()

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /home/alex/.tradingstrategy


## Run the backtest

In [12]:
import logging

from tradeexecutor.backtest.backtest_runner import run_backtest_inline

state, universe, debug_dump = run_backtest_inline(
    name="ETH/USDC fast and slow EMA example",
    start_at=START_AT,
    end_at=END_AT,
    client=client,
    cycle_duration=TRADING_STRATEGY_CYCLE,
    decide_trades=decide_trades,
    universe=universe,
    # create_trading_universe=create_single_pair_trading_universe,
    initial_deposit=INITIAL_DEPOSIT,
    reserve_currency=RESERVE_CURRENCY,
    trade_routing=TRADE_ROUTING,
    log_level=logging.WARNING,
    engine_version=TRADING_STRATEGY_TYPE_ENGINE_VERSION,
)

trade_count = len(list(state.portfolio.get_all_trades()))
print(f"Backtesting completed, backtested strategy made {trade_count} trades")

  0%|          | 0/24192000 [00:00<?, ?it/s]

Backtesting completed, backtested strategy made 32 trades


In [13]:
print(f"Positions taken: {len(list(state.portfolio.get_all_positions()))}")
print(f"Trades made: {len(list(state.portfolio.get_all_trades()))}")

Positions taken: 16
Trades made: 32


In [14]:
from tradeexecutor.visual.single_pair import visualise_single_pair, visualise_single_pair_positions_with_duration_and_slippage
from tradingstrategy.charting.candle_chart import VolumeBarMode

figure = visualise_single_pair(
    state,
    universe.universe.candles,
    start_at=START_AT,
    end_at=END_AT,
    volume_bar_mode=VolumeBarMode.hidden,
    volume_axis_name="Volume (USD)",
    height = 1000,
    hover_text=True,
)

figure.show()

/home/alex/work/tradingstrategy-ai/stuff/trade-executor/tradeexecutor/visual/utils.py:243: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [15]:
from tradeexecutor.visual.benchmark import visualise_benchmark

traded_pair = universe.universe.pairs.get_single()

fig = visualise_benchmark(
    state.name,
    portfolio_statistics=state.stats.portfolio,
    all_cash=state.portfolio.get_initial_deposit(),
    buy_and_hold_asset_name=traded_pair.base_token_symbol,
    buy_and_hold_price_series=universe.universe.candles.get_single_pair_data()["close"],
    start_at=START_AT,
    end_at=END_AT,
    height=800
)

fig.show()

In [16]:
from tradeexecutor.analysis.trade_analyser import build_trade_analysis

analysis = build_trade_analysis(state.portfolio)

In [17]:
from IPython.core.display_functions import display

summary = analysis.calculate_all_summary_stats_by_side(state=state, time_bucket=CANDLE_TIME_BUCKET)

with pd.option_context("display.max_row", None):
    display(summary)

,All,Long,Short
Trading period length,239 days 20 hours,-,-
Return %,11.14%,18.17%,0.00%
Annualised return %,16.96%,27.65%,-
Cash at start,"$50,000.00",-,-
Value at end,"$55,571.14",-,-
Trade volume,"$1,343,461.06","$1,343,461.06",$0.00
Position win percent,50.00%,50.00%,-
Total positions,16,16,0
Won positions,8,8,0
Lost positions,8,8,0


In [18]:
from IPython.core.display_functions import display

summary = analysis.calculate_summary_statistics(state=state, time_bucket=CANDLE_TIME_BUCKET)

# with pd.option_context("display.max_row", None):
#      display(summary.to_dataframe())

summary.display()

,
Returns,
Annualised return %,16.96%
Lifetime return %,11.14%
Realised PnL,"$5,571.14"
Unrealised PnL,$0.00
Trade period,239 days 20 hours
,
Holdings,
Total assets,"$55,571.14"
Cash left,"$55,571.14"
